## Setup

In [ ]:
# Load packages
using DifferentialEquations
using Plots, StatsPlots
using CSV, DataFrames
using Turing
using LaTeXStrings
using XLSX
using Optim
using StatsBase
using Random
using KernelDensity
using ProgressMeter
using Distributions
using Measures

# Setup plots standard
Plots.default(fontfamily = ("computer modern"))

# Set seed 
Random.seed!(42)

In [ ]:
# Add rhs file
include("model_rhs.jl")
    
# Add parameter file
include("model_default_param.jl")

# Add treatment rhs
include("model_rhs_treatment.jl")

# Add function for calculating VAF
include("model_calc_VAF.jl")

# Add model for Turing inference - individual
include("model_infer_dynamics.jl")

# Add model for Turing inference - hierarchical
include("model_infer_dynamics_Hierarchical_Gibbs_DALIAH.jl")

In [ ]:
# Setup initial conditions for master curve
x00 = 1.0*10^5
x10 = 2.5*10^6
x20 = 6.4*10^11
y00 = 1
y10 = 0
y20 = 0
a0 = 8.1*10^2
s0 = 1

# Collect in one vector
u0 = [x00,x10,x20,y00,y10,y20,a0,s0]

# Setup and solve ODEproblem
tspan = (0.0,90*365)
prob = ODEProblem(model_rhs, u0, tspan, p)
sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

# Save as named tuple
master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

In [ ]:
# Setup dataframes
df_D = DataFrame

# Load data
path = "M:/data_cancer/DALIAH/Trine_DALIAH_5y/DALIAH_5y - Export for Julia.csv"
df_D = CSV.read(path, df_D)

# Change data types
df_D[!,:days] = convert.(Float64,df_D[:,:days])
df_D[!,:RUX] = convert.(Float64,df_D[:,:RUX])


df_D

In [ ]:
# Convert to starting at time for first data point - no matter if JAK or treatment
for i in unique(df_D.patientID)
    # Set pID
    pID = i
    
    # Extract minimum number of days - where IFN>0 or JAK>=0
    df_p = df_D[df_D.patientID .== pID,:]

    minDays = minimum(df_p.days[df_p.IFN .> 0  .|| df_p.JAK .>= 0, :])
    
    # Subtract minimum number of days
    df_D.days[df_D.patientID .== pID, :] = df_D.days[df_D.patientID .== pID, :] .- minDays
end
df_D

## Test a single patient

In [ ]:
# Setup list of models to compare
model_list = ["py0", "py1", "py0py1", "dy0", "dy1IFN", "dy0dy1", "py0dy0", "py0dy1", "dy0py1", "py1dy1",
              "py0py1dy0", "py0py1dy1", "py1dy0dy1", "py0dy0dy1", "py0py1dy0dy1"]

In [ ]:
# Choose treatment effect
effect = model_list[8]


# Extract number of parameterss
if effect == "sy0dy1" || effect == "py0py1" || effect == "dy0dy1" || effect == "py0dy0" || effect == "py0dy1" || effect == "dy0py1" || effect == "py1dy1"
    n_param = 4
elseif effect == "dy1" || effect == "py0" || effect == "py1" || effect == "dy0" || effect == "dy1IFN"
    n_param = 3
elseif effect == "py0py1dy0" || effect == "py0py1dy1" || effect == "py1dy0dy1" || effect == "py0dy0dy1"
    n_param = 5
elseif effect == "py0py1dy0dy1"
    n_param = 6
end

# Storage
if n_param == 4
    df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], sigma = Float64[], 
                       loglik = Float64[], AIC = Float64[], BIC = Float64[])
elseif n_param == 3
    df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[],
                       AIC = Float64[], BIC = Float64[])
elseif n_param == 5
    df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], initJAK = Float64[], 
                       sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
elseif n_param == 6
    df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], rho4 = Float64[],
                       initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
end

# Extract relevant data
pID = 2
df_p = df_D[df_D.patientID .== pID, :]

# Extract number of data points
n = length(df_p.JAK)

# Extract VAF - but remove NAN-observations
pVAF = df_p.JAK
pVAF = pVAF[.!isnan.(pVAF)]

# Choose to estimate initial JAK
estInitJAK = true

# Instantiate model
model = model_infer_dynamics(pVAF,df_p,effect,p,master_curve,estInitJAK)

# Set seed 
Random.seed!(42)

# Storage
if n_param == 4
    df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], 
                            sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
elseif n_param == 3
    df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], 
                            sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
elseif n_param == 5
    df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                            initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
elseif n_param == 6
    df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                            rho4 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[], 
                            AIC = Float64[], BIC = Float64[])
end

# Generate a MLE estimate - loop
for j=1:10
    # Optimise likelihood
    mle_estimate = maximum_likelihood(model, ParticleSwarm(); maxiters = 100, allow_f_increases=true)

    # Setup temporary dataframe and save
    if n_param == 4
        df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
    elseif n_param == 3
        df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
    elseif n_param == 5
        df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                            rho3 = coef(mle_estimate)[:rho3],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
    elseif n_param == 6
        df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                            rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
    end
    loglik = loglikelihood(model, df_temp)
    if n_param == 4
        df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                            loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
    elseif n_param == 3
        df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                            loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
    elseif n_param == 5
        df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                            rho3 = coef(mle_estimate)[:rho3],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                            loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
    elseif n_param == 6
        df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                            rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                            initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                            loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
    end
    append!(df_MLE_temp, df_temp)
end



In [ ]:
# Extract relevant data
pID = 2
df_p = df_D[df_D.patientID .== pID, :]

# Extract latest data point
maxDays = maximum(df_p.days)

# Extract VAF - but remove NAN-observations
pVAF = df_p.JAK
pVAF = pVAF[.!isnan.(pVAF)]

# Choose to estimate initial JAK
estInitJAK = true

# Choose time for calculating VAF
pDays = collect(1:11*365)
pDays2 = df_p.days[.!isnan.(df_p.JAK)]

# Load standard parameters
include("model_default_param.jl")

# Extract MLE estimate
df_MLE_p = df_MLE_temp
if n_param == 4
    rho = [df_MLE_p.rho1[1], df_MLE_p.rho2[1]]
elseif n_param == 3
    rho = df_MLE_p.rho1[1]
elseif n_param == 5
    rho = [df_MLE_p.rho1[1], df_MLE_p.rho2[1], df_MLE_p.rho3[1]]
elseif n_param == 6
    rho = [df_MLE_p.rho1[1], df_MLE_p.rho2[1], df_MLE_p.rho3[1], df_MLE_p.rho4[1]]
end
initJAK = df_MLE_p.initJAK[1]

# Calculate VAF using function
VAF_MLE, sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
VAF_MLE_points, sol = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)

# Calculate RMSE
RMSE = rmsd(VAF_MLE_points,pVAF)

# Plot results
figVAF = plot(pDays[pDays.<=maxDays]/30.4, VAF_MLE[pDays.<=maxDays]*100, linewidth = 3, label = L"JAK2"*" VAF - MLE", colour= :green1)
plot!(pDays[pDays.>maxDays]/30.4, VAF_MLE[pDays.>maxDays]*100, linewidth = 3, label = L"JAK2"*" VAF - MLE", colour= :green1, linestyle = :dash)
# plot!(pDays/30.4, VAF_true*100, linewidth = 3, label = L"JAK2"*" VAF - True", colour = :darkgoldenrod2)
scatter!(df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=4)
if n_param == 4
    title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho_{p_{y_0}}="
           *"$(round(df_MLE_p.rho1[1]; sigdigits=3)), " * L"\rho_{p_{y_1}}="*"$(round(df_MLE_p.rho2[1]; sigdigits=3))")
elseif n_param == 3
    title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho_{p_{y_0}}="
           *"$(round(df_MLE_p.rho1[1]; sigdigits=3))")
end
xlabel!(L"t"*"/months")
ylabel!(L"JAK2"*" VAF/%")
t_end = 125
xlims!(0, t_end)
ylims!(0,100)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
    legend = :none)
figIFN = plot(df_p.days/30.4, df_p.IFN, line =:steppost, linewidth = 3, label = "", colour= :blue1)
plot!([maxDays/30.4,120], [df_p.IFN[end],df_p.IFN[end]], line =:steppost, linewidth = 3, label = "", colour= :blue1,
       linestyle = :dash)
title!("IFN Dose for Patient $(pID)")
xlabel!(L"t"*"/months")
ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
xlims!(0, t_end)
ylims!(0, 20)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10)
figcombined = plot(figVAF,figIFN, layout=grid(2,1, heights=(7/10, 3/10)), link =:x, size = (700, 500), margin=5mm)

## Sensitivity analysis for selected patients

In [ ]:
indices = vcat(1:1:12,15:1:28)
symbol_list = [:alphax0, :alphay0, :px0, :py0, :alphax1, :alphay1, :px1, :py1, :Ax0, :Ay0, :Ax1, :Ay1, :dx1, :dy1,
         :dx2, :dy2, :cxx, :cyx, :cxy, :cyy, :es, :rs, :ea, :I, :sx0, :sy0]
text_list = ["No",L"\alpha_{x_0}", L"\alpha_{y_0}", L"p_{x_0}", L"p_{y_0}", L"\alpha_{x_1}", L"\alpha_{y_1}", L"p_{x_1}", 
             L"p_{y_1}", L"A_{x_0}", L"A_{y_0}", L"A_{x_1}", L"A_{y_1}", L"d_{x_1}", L"d_{y_1}", L"d_{x_2}",
             L"d_{y_2}", L"c_{xx}", L"c_{yx}", L"c_{xy}", L"c_{yy}", L"e_s", L"r_s", L"e_a", L"I", L"s_{x_0}", L"s_{y_0}"]

In [ ]:
# Change a parameter
field = symbol_list[3]
temp_p = merge(p, [field=>1.1*p[indices[3]]])

# Calculate new master_curve
# Setup and solve ODEproblem
tspan = (0.0,150*365)
prob = ODEProblem(model_rhs, u0, tspan, temp_p)
sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

# Save as named tuple
temp_master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

In [ ]:
# Choose treatment effect
effect = "py0dy1"


# Extract number of parameterss
if effect == "sy0dy1" || effect == "py0py1" || effect == "dy0dy1" || effect == "py0dy0" || effect == "py0dy1" || effect == "dy0py1" || effect == "py1dy1"
    n_param = 4
elseif effect == "dy1" || effect == "py0" || effect == "py1" || effect == "dy0" || effect == "dy1IFN"
    n_param = 3
elseif effect == "py0py1dy0" || effect == "py0py1dy1" || effect == "py1dy0dy1" || effect == "py0dy0dy1"
    n_param = 5
elseif effect == "py0py1dy0dy1"
    n_param = 6
end

# Storage
if n_param == 4
    df_MLE_inc = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], sigma = Float64[], 
                       loglik = Float64[], AIC = Float64[], BIC = Float64[])
    df_MLE_dec = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], sigma = Float64[], 
                       loglik = Float64[], AIC = Float64[], BIC = Float64[])
elseif n_param == 3
    df_MLE_inc = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[],
                       AIC = Float64[], BIC = Float64[])
    df_MLE_dec = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[],
                       AIC = Float64[], BIC = Float64[])
elseif n_param == 5
    df_MLE_inc = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], initJAK = Float64[], 
                       sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
    df_MLE_dec = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], initJAK = Float64[], 
                       sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
elseif n_param == 6
    df_MLE_inc = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], rho4 = Float64[],
                       initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
    df_MLE_dec = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], rho4 = Float64[],
                       initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
end

# Choose patients to investigate
p_list = [3, 76, 110]

# Loop over patients 
for k=1:length(p_list)

    # Extract relevant data
    pID = p_list[k]
    df_p = df_D[df_D.patientID .== pID, :]

    # Extract number of data points
    n = length(df_p.JAK)

    # Extract VAF - but remove NAN-observations
    pVAF = df_p.JAK
    pVAF = pVAF[.!isnan.(pVAF)]

    # Choose to estimate initial JAK
    estInitJAK = true

    # Run one without changing any parameters

    # Change a parameter
    field = symbol_list[1]
    temp_p = merge(p, [field=>1.0*p[indices[1]]])

    # Calculate new master_curve
    # Setup and solve ODEproblem
    tspan = (0.0,150*365)
    prob = ODEProblem(model_rhs, u0, tspan, temp_p)
    sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

    # Save as named tuple
    temp_master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
    a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

    # Instantiate model
    model = model_infer_dynamics(pVAF,df_p,effect,temp_p,temp_master_curve,estInitJAK)

    # Set seed 
    Random.seed!(42)

    # Storage
    if n_param == 4
        df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], 
                                sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
    elseif n_param == 3
        df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], 
                                sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
    elseif n_param == 5
        df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
    elseif n_param == 6
        df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                rho4 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[], 
                                AIC = Float64[], BIC = Float64[])
    end

    # Generate a MLE estimate - loop
    for j=1:10
        # Optimise likelihood
        mle_estimate = maximum_likelihood(model, ParticleSwarm(); maxiters = 100, allow_f_increases=true)

        # Setup temporary dataframe and save
        if n_param == 4
            df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
        elseif n_param == 3
            df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
        elseif n_param == 5
            df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                rho3 = coef(mle_estimate)[:rho3],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
        elseif n_param == 6
            df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
        end
        loglik = loglikelihood(model, df_temp)
        if n_param == 4
            df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
        elseif n_param == 3
            df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
        elseif n_param == 5
            df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                rho3 = coef(mle_estimate)[:rho3],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
        elseif n_param == 6
            df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
        end
        append!(df_MLE_temp, df_temp)
    end

    # Save best MLE
    push!(df_MLE_inc, df_MLE_temp[argmax(df_MLE_temp.loglik),:])
    push!(df_MLE_dec, df_MLE_temp[argmax(df_MLE_temp.loglik),:])

    @showprogress dt=1 desc="Running all parameter changes..." for i=1:length(indices)
         # Change a parameter
        field = symbol_list[i]
        temp_p = merge(p, [field=>1.1*p[indices[i]]])

        # Calculate new master_curve
        # Setup and solve ODEproblem
        tspan = (0.0,150*365)
        prob = ODEProblem(model_rhs, u0, tspan, temp_p)
        sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

        # Save as named tuple
        temp_master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
        a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

        # Instantiate model
        model = model_infer_dynamics(pVAF,df_p,effect,temp_p,temp_master_curve,estInitJAK)

        # Set seed 
        Random.seed!(42)

        # Storage
        if n_param == 4
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], 
                                    sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 3
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], 
                                    sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 5
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                    initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 6
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                    rho4 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[], 
                                    AIC = Float64[], BIC = Float64[])
        end

        # Generate a MLE estimate - loop
        for j=1:10
            # Optimise likelihood
            mle_estimate = maximum_likelihood(model, ParticleSwarm(); maxiters = 100, allow_f_increases=true)

            # Setup temporary dataframe and save
            if n_param == 4
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 3
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 5
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 6
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            end
            loglik = loglikelihood(model, df_temp)
            if n_param == 4
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 3
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 5
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 6
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            end
            append!(df_MLE_temp, df_temp)
        end

        # Save best MLE
        push!(df_MLE_inc, df_MLE_temp[argmax(df_MLE_temp.loglik),:])
        
        # Change a parameter
        field = symbol_list[i]
        temp_p = merge(p, [field=>0.9*p[indices[i]]])

        # Calculate new master_curve
        # Setup and solve ODEproblem
        tspan = (0.0,150*365)
        prob = ODEProblem(model_rhs, u0, tspan, temp_p)
        sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

        # Save as named tuple
        temp_master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
        a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

        # Instantiate model
        model = model_infer_dynamics(pVAF,df_p,effect,temp_p,temp_master_curve,estInitJAK)

        # Set seed 
        Random.seed!(42)

        # Storage
        if n_param == 4
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], 
                                    sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 3
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], 
                                    sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 5
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                    initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 6
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                    rho4 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[], 
                                    AIC = Float64[], BIC = Float64[])
        end

        # Generate a MLE estimate - loop
        for j=1:10
            # Optimise likelihood
            mle_estimate = maximum_likelihood(model, ParticleSwarm(); maxiters = 100, allow_f_increases=true)

            # Setup temporary dataframe and save
            if n_param == 4
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 3
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 5
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 6
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            end
            loglik = loglikelihood(model, df_temp)
            if n_param == 4
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 3
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 5
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 6
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            end
            append!(df_MLE_temp, df_temp)
        end

        # Save best MLE
        push!(df_MLE_dec, df_MLE_temp[argmax(df_MLE_temp.loglik),:])
    end
end


In [ ]:
df_MLE_dec

In [ ]:
# Save results
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/df_MLE_sensitivity_inc.csv"
CSV.write(path, df_MLE_inc)
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/df_MLE_sensitivity_dec.csv"
CSV.write(path, df_MLE_dec)

In [ ]:
# Load results
df_MLE_inc = DataFrame
df_MLE_dec = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/df_MLE_sensitivity_inc.csv"
df_MLE_inc = CSV.read(path, df_MLE_inc)
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/df_MLE_sensitivity_dec.csv"
df_MLE_dec = CSV.read(path, df_MLE_dec)

In [ ]:
# Choose patients to investigate
p_list = [3, 76, 110]
P = length(p_list)
S = length(symbol_list)

# Storage
VAF_after_2_years_inc = zeros(S+1,P)
VAF_after_2_years_dec = zeros(S+1,P)
time_to_reduce_VAF_06_inc = zeros(S+1,P)
time_to_reduce_VAF_06_dec = zeros(S+1,P)
RMSE_inc = zeros(S+1,P)
RMSE_dec = zeros(S+1,P)

# Initialise plots
plot_inc_3 = Any[]
plot_dec_3 = Any[]
plot_inc_76 = Any[]
plot_dec_76 = Any[]
plot_inc_110 = Any[]
plot_dec_110 = Any[]


# Choose treatment effect
effect = "py0dy1"

# Extract number of parameterss
if effect == "sy0dy1" || effect == "py0py1" || effect == "dy0dy1" || effect == "py0dy0" || effect == "py0dy1" || effect == "dy0py1" || effect == "py1dy1"
    n_param = 4
elseif effect == "dy1" || effect == "py0" || effect == "py1" || effect == "dy0" || effect == "dy1IFN"
    n_param = 3
elseif effect == "py0py1dy0" || effect == "py0py1dy1" || effect == "py1dy0dy1" || effect == "py0dy0dy1"
    n_param = 5
elseif effect == "py0py1dy0dy1"
    n_param = 6
end

# Loop over patients 
for k=1:length(p_list)

    # Choose patient 
    pID = p_list[k]
    df_p = df_D[df_D.patientID .== pID, :]

    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF - but remove NAN-observations
    pVAF = df_p.JAK
    pVAF = pVAF[.!isnan.(pVAF)]

    # Choose to estimate initial JAK
    estInitJAK = true

     # Choose time for calculating VAF
    pDays = 0:1:75*30.4
    pDays_plot = pDays[1:30:end]
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]
    
    # Extract MLE estimate
    df_MLE_p_inc = df_MLE_inc[df_MLE_inc.patientID .== pID,:]
    df_MLE_p_dec = df_MLE_dec[df_MLE_dec.patientID .== pID,:]
    if n_param == 4
        rho = [df_MLE_p_inc.rho1[1], df_MLE_p_inc.rho2[1]]
    elseif n_param == 3
        rho = df_MLE_p_inc.rho1[1]
    elseif n_param == 5
        rho = [df_MLE_p_inc.rho1[1], df_MLE_p_inc.rho2[1], df_MLE_p_inc.rho3[1]]
    elseif n_param == 6
        rho = [df_MLE_p_inc.rho1[1], df_MLE_p_inc.rho2[1], df_MLE_p_inc.rho3[1], df_MLE_p_inc.rho4[1]]
    end
    initJAK = df_MLE_p_inc.initJAK[1]
    
    # Simulate for base parameters
    VAF, sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
    VAF_MLE_points, sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
    VAF_plot = VAF[1:30:end]
    
    # Calculate RMSE
    RMSE = rmsd(VAF_MLE_points,pVAF)
    RMSE_inc[1,k] = RMSE
    RMSE_dec[1,k] = RMSE
    
     # Extract VAF after 2 years
    VAF_after_2_years_inc[1,k] = VAF[2*365+1]
    VAF_after_2_years_dec[1,k] = VAF[2*365+1]
    
    # Extract time to reduction in VAF by 60%
    diffVAF = VAF .- 0.6*initJAK
    ID = findfirst(x -> x<0, diffVAF)
        
    # Save time
    time_to_reduce_VAF_06_inc[1,k] = sol.t[ID]
    time_to_reduce_VAF_06_dec[1,k] = sol.t[ID]
    
    # Plot results
    figVAF_stamp = plot()
    xlims!(0, 75)
    ylims!(0, 100)
    # ylabel!(L"JAK2"*" VAF [%]")
    title!(text_list[1])
    annotate!(50, 90, text(L"\textrm{RMSE}=" * "$(round(RMSE; sigdigits=3))", :center, 15))
    annotate!(50, 65, text(L"\rho="*"$(round(rho[1]; sigdigits=3))", :center, 15))
    annotate!(50, 40, text(L"\delta="*"$(round(rho[2]; sigdigits=3))", :center, 15))
    p2 = twinx()
    plot!(p2, df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
        linewidth = 3, label = "", colour= :blue1)
    plot!(p2, [maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
      linewidth = 3, label = "", colour= :blue1)
    # xlabel!(p2, L"t"*" [months]")
    # ylabel!(p2, "IFN"*L"-$\alpha$ $\left[\mathrm{\mu g/day}\right]$")
    xlims!(p2, 0, 75)
    ylims!(p2, 0, 20)
    plot!(pDays_plot/30.4, VAF_plot*100, linewidth = 3, colour = :black)
    scatter!(df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.12)
    figVAF_stamp = plot(figVAF_stamp, size = (700, 500), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,legend = :none)
    if k==1
        push!(plot_inc_3,figVAF_stamp)
        push!(plot_dec_3,figVAF_stamp)
    elseif k==2
        push!(plot_inc_76,figVAF_stamp)
        push!(plot_dec_76,figVAF_stamp)
    elseif k==3
        push!(plot_inc_110,figVAF_stamp)
        push!(plot_dec_110,figVAF_stamp)
    end
    
    # Loop over all parameters
    @showprogress dt=1 desc="Running all parameter changes..." for i=1:length(indices)
         # Increase a parameter
        field = symbol_list[i]
        temp_p = merge(p, [field=>1.1*p[indices[i]]])

        # Calculate new master_curve
        # Setup and solve ODEproblem
        tspan = (0.0,150*365)
        prob = ODEProblem(model_rhs, u0, tspan, temp_p)
        sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

        # Save as named tuple
        temp_master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
        a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
        
        # Extract MLE estimate
        if n_param == 4
            rho = [df_MLE_p_inc.rho1[i+1], df_MLE_p_inc.rho2[i+1]]
        elseif n_param == 3
            rho = df_MLE_p_inc.rho1[i+1]
        elseif n_param == 5
            rho = [df_MLE_p_inc.rho1[i+1], df_MLE_p_inc.rho2[i+1], df_MLE_p_inc.rho3[i+1]]
        elseif n_param == 6
            rho = [df_MLE_p_inc.rho1[i+1], df_MLE_p_inc.rho2[i+1], df_MLE_p_inc.rho3[i+1], df_MLE_p_inc.rho4[i+1]]
        end
        initJAK = df_MLE_p_inc.initJAK[i+1]
        
        # Calculate VAF
        VAF, sol = model_calc_VAF(rho,df_p,effect,pDays,temp_p,temp_master_curve,initJAK)
        VAF_MLE_points, sol_points = model_calc_VAF(rho,df_p,effect,pDays2,temp_p,temp_master_curve,initJAK)
        VAF_plot = VAF[1:30:end]
        
        # Calculate RMSE
        RMSE = rmsd(VAF_MLE_points,pVAF)
        RMSE_inc[i+1,k] = RMSE
        
        # Extract VAF after 2 years
        VAF_after_2_years_inc[i+1,k] = VAF[2*365+1]
    
        # Extract time to reduction in VAF by half
        diffVAF = VAF .- 0.6*initJAK
        ID = findfirst(x -> x<0, diffVAF)
        
        # Save time
        time_to_reduce_VAF_06_inc[i+1,k] = sol.t[ID]
        
        # Plot results
        figVAF_stamp = plot()
        xlims!(0, 75)
        ylims!(0, 100)
        # ylabel!(L"JAK2"*" VAF [%]")
        title!(L"1.1\cdot"*text_list[i+1])
        annotate!(50, 90, text(L"\textrm{RMSE}=" * "$(round(RMSE; sigdigits=3))", :center, 15))
        annotate!(50, 65, text(L"\rho="*"$(round(rho[1]; sigdigits=3))", :center, 15))
        annotate!(50, 40, text(L"\delta="*"$(round(rho[2]; sigdigits=3))", :center, 15))
        p2 = twinx()
        plot!(p2, df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
        plot!(p2, [maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
        # xlabel!(p2, L"t"*" [months]")
        # ylabel!(p2, "IFN"*L"-$\alpha$ $\left[\mathrm{\mu g/day}\right]$")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 20)
        plot!(pDays_plot/30.4, VAF_plot*100, linewidth = 3, colour = :black)
        scatter!(df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.12)
        figVAF_stamp = plot(figVAF_stamp, size = (700, 500), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
        plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
        if k==1
            push!(plot_inc_3,figVAF_stamp)
        elseif k==2
            push!(plot_inc_76,figVAF_stamp)
        elseif k==3
            push!(plot_inc_110,figVAF_stamp)
        end
        
        # Decrease a parameter
        field = symbol_list[i]
        temp_p = merge(p, [field=>0.9*p[indices[i]]])

        # Calculate new master_curve
        # Setup and solve ODEproblem
        tspan = (0.0,150*365)
        prob = ODEProblem(model_rhs, u0, tspan, temp_p)
        sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

        # Save as named tuple
        temp_master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
        a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
        
        # Extract MLE estimate
        if n_param == 4
            rho = [df_MLE_p_dec.rho1[i+1], df_MLE_p_dec.rho2[i+1]]
        elseif n_param == 3
            rho = df_MLE_p_dec.rho1[i+1]
        elseif n_param == 5
            rho = [df_MLE_p_dec.rho1[i+1], df_MLE_p_dec.rho2[i+1], df_MLE_p_dec.rho3[i+1]]
        elseif n_param == 6
            rho = [df_MLE_p_dec.rho1[i+1], df_MLE_p_dec.rho2[i+1], df_MLE_p_dec.rho3[i+1], df_MLE_p_dec.rho4[i+1]]
        end
        initJAK = df_MLE_p_inc.initJAK[i+1]
        
        # Calculate VAF
        VAF, sol = model_calc_VAF(rho,df_p,effect,pDays,temp_p,temp_master_curve,initJAK)
        VAF_MLE_points, sol_points = model_calc_VAF(rho,df_p,effect,pDays2,temp_p,temp_master_curve,initJAK)
        VAF_plot = VAF[1:30:end]
        
        # Calculate RMSE
        RMSE = rmsd(VAF_MLE_points,pVAF)
        RMSE_dec[i+1,k] = RMSE
       
        # Extract VAF after 2 years
        VAF_after_2_years_dec[i+1,k] = VAF[2*365+1]
    
        # Extract time to reduction in VAF by half
        diffVAF = VAF .- 0.6*initJAK
        ID = findfirst(x -> x<0, diffVAF)
    
        # Save time
        time_to_reduce_VAF_06_dec[i+1,k] = sol.t[ID]
        
        # Plot results
        figVAF_stamp = plot()
        xlims!(0, 75)
        ylims!(0, 100)
        # ylabel!(L"JAK2"*" VAF [%]")
        title!(L"0.9\cdot"*text_list[i+1])
        annotate!(50, 90, text(L"\textrm{RMSE}=" * "$(round(RMSE; sigdigits=3))", :center, 15))
        annotate!(50, 65, text(L"\rho="*"$(round(rho[1]; sigdigits=3))", :center, 15))
        annotate!(50, 40, text(L"\delta="*"$(round(rho[2]; sigdigits=3))", :center, 15))
        p2 = twinx()
        plot!(p2, df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
        plot!(p2, [maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
        # xlabel!(p2, L"t"*" [months]")
        # ylabel!(p2, "IFN"*L"-$\alpha$ $\left[\mathrm{\mu g/day}\right]$")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 20)
        # plot!(pDays_plot/30.4, VAF_plot*100, linewidth = 3, colour = :black)
        plot!(sol.t/30.4, 100*sol[6,:] ./ (sol[3,:]+sol[6,:]), linewidth = 3, colour = :black)
        scatter!(df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.12)
        figVAF_stamp = plot(figVAF_stamp, size = (700, 500), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
        plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,legend = :none)
        if k==1
            push!(plot_dec_3,figVAF_stamp)
        elseif k==2
            push!(plot_dec_76,figVAF_stamp)
        elseif k==3
            push!(plot_dec_110,figVAF_stamp)
        end
    end
end

In [ ]:
fig_inc_3 = plot(plot_inc_3..., layout= (7,4), size = (2000, 1600), margin=8mm, leftmargin = 8mm, rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_inc_3,figpath*"sensitivity_inc_3.png")
savefig(fig_inc_3,figpath*"sensitivity_inc_3.pdf")
savefig(fig_inc_3,figpath*"sensitivity_inc_3.svg")

In [ ]:
fig_dec_3 = plot(plot_dec_3..., layout= (7,4), size = (2000, 1600), margin=8mm, leftmargin = 8mm, rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_dec_3,figpath*"sensitivity_dec_3.png")
savefig(fig_dec_3,figpath*"sensitivity_dec_3.pdf")
savefig(fig_dec_3,figpath*"sensitivity_dec_3.svg")

In [ ]:
fig_inc_76 = plot(plot_inc_76..., layout= (7,4), size = (2000, 1600), margin=8mm, leftmargin = 8mm, rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_inc_76,figpath*"sensitivity_inc_76.png")
savefig(fig_inc_76,figpath*"sensitivity_inc_76.pdf")
savefig(fig_inc_76,figpath*"sensitivity_inc_76.svg")

In [ ]:
fig_dec_76 = plot(plot_dec_76..., layout= (7,4), size = (2000, 1600), margin=8mm, leftmargin = 8mm, rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_dec_76,figpath*"sensitivity_dec_76.png")
savefig(fig_dec_76,figpath*"sensitivity_dec_76.pdf")
savefig(fig_dec_76,figpath*"sensitivity_dec_76.svg")

In [ ]:
fig_inc_110 = plot(plot_inc_110..., layout= (7,4), size = (2000, 1600), margin=8mm, leftmargin = 8mm, rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_inc_110,figpath*"sensitivity_inc_110.png")
savefig(fig_inc_110,figpath*"sensitivity_inc_110.pdf")
savefig(fig_inc_110,figpath*"sensitivity_inc_110.svg")

In [ ]:
fig_dec_110 = plot(plot_dec_110..., layout= (7,4), size = (2000, 1600), margin=8mm, leftmargin = 8mm, rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_dec_110,figpath*"sensitivity_dec_110.png")
savefig(fig_dec_110,figpath*"sensitivity_dec_110.pdf")
savefig(fig_dec_110,figpath*"sensitivity_dec_110.svg")

In [ ]:
df_MLE_inc

In [ ]:
df_MLE_inc_p = df_MLE_inc[df_MLE_inc.patientID .== 110,:]
df_MLE_dec_p = df_MLE_dec[df_MLE_dec.patientID .== 110,:]
fig_rho_inc_110 = plot()
bar!((df_MLE_inc_p.rho1 ./ df_MLE_inc_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.110)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_rho_dec_110 = plot()
bar!((df_MLE_dec_p.rho1 ./ df_MLE_dec_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.110)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_delta_inc_110 = plot()
bar!((df_MLE_inc_p.rho2 ./ df_MLE_inc_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.110)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_delta_dec_110 = plot()
bar!((df_MLE_dec_p.rho2 ./ df_MLE_dec_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.110)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

In [ ]:
fig_bar_VAF_inc_3 = plot()
bar!((VAF_after_2_years_inc[:,1] ./ VAF_after_2_years_inc[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,1.3)
plot!(size = (1000, 500), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_VAF_dec_3 = plot()
bar!((VAF_after_2_years_dec[:,1] ./ VAF_after_2_years_dec[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_time_inc_3 = plot()
bar!((time_to_reduce_VAF_06_inc[:,1] ./ time_to_reduce_VAF_06_inc[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_time_dec_3 = plot()
bar!((time_to_reduce_VAF_06_dec[:,1] ./ time_to_reduce_VAF_06_dec[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_RMSE_inc_3 = plot()
bar!((RMSE_inc[:,1] ./ RMSE_inc[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_RMSE_dec_3 = plot()
bar!((RMSE_dec[:,1] ./ RMSE_dec[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

df_MLE_inc_p = df_MLE_inc[df_MLE_inc.patientID .== 3,:]
df_MLE_dec_p = df_MLE_dec[df_MLE_dec.patientID .== 3,:]
fig_bar_rho_inc_3 = plot()
bar!((df_MLE_inc_p.rho1 ./ df_MLE_inc_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_rho_dec_3 = plot()
bar!((df_MLE_dec_p.rho1 ./ df_MLE_dec_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_delta_inc_3 = plot()
bar!((df_MLE_inc_p.rho2 ./ df_MLE_inc_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_delta_dec_3 = plot()
bar!((df_MLE_dec_p.rho2 ./ df_MLE_dec_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 3")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_VAF_inc_76 = plot()
bar!((VAF_after_2_years_inc[:,2] ./ VAF_after_2_years_inc[1,2]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_VAF_dec_76 = plot()
bar!((VAF_after_2_years_dec[:,2] ./ VAF_after_2_years_dec[1,2]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_time_inc_76 = plot()
bar!((time_to_reduce_VAF_06_inc[:,2] ./ time_to_reduce_VAF_06_inc[1,2]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_time_dec_76 = plot()
bar!((time_to_reduce_VAF_06_dec[:,2] ./ time_to_reduce_VAF_06_dec[1,2]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_RMSE_inc_76 = plot()
bar!((RMSE_inc[:,1] ./ RMSE_inc[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_RMSE_dec_76 = plot()
bar!((RMSE_dec[:,1] ./ RMSE_dec[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

df_MLE_inc_p = df_MLE_inc[df_MLE_inc.patientID .== 76,:]
df_MLE_dec_p = df_MLE_dec[df_MLE_dec.patientID .== 76,:]
fig_bar_rho_inc_76 = plot()
bar!((df_MLE_inc_p.rho1 ./ df_MLE_inc_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_rho_dec_76 = plot()
bar!((df_MLE_dec_p.rho1 ./ df_MLE_dec_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_delta_inc_76 = plot()
bar!((df_MLE_inc_p.rho2 ./ df_MLE_inc_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_delta_dec_76 = plot()
bar!((df_MLE_dec_p.rho2 ./ df_MLE_dec_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 76")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_VAF_inc_110 = plot()
bar!((VAF_after_2_years_inc[:,3] ./ VAF_after_2_years_inc[1,3]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_VAF_dec_110 = plot()
bar!((VAF_after_2_years_dec[:,3] ./ VAF_after_2_years_dec[1,3]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_time_inc_110 = plot()
bar!((time_to_reduce_VAF_06_inc[:,3] ./ time_to_reduce_VAF_06_inc[1,3]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_time_dec_110 = plot()
bar!((time_to_reduce_VAF_06_dec[:,3] ./ time_to_reduce_VAF_06_dec[1,3]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.3)
plot!(size = (1000, 500), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_RMSE_inc_110 = plot()
bar!((RMSE_inc[:,1] ./ RMSE_inc[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_RMSE_dec_110 = plot()
bar!((RMSE_dec[:,1] ./ RMSE_dec[1,1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,1.3)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

df_MLE_inc_p = df_MLE_inc[df_MLE_inc.patientID .== 110,:]
df_MLE_dec_p = df_MLE_dec[df_MLE_dec.patientID .== 110,:]
fig_bar_rho_inc_110 = plot()
bar!((df_MLE_inc_p.rho1 ./ df_MLE_inc_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_rho_dec_110 = plot()
bar!((df_MLE_dec_p.rho1 ./ df_MLE_dec_p.rho1[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_delta_inc_110 = plot()
bar!((df_MLE_inc_p.rho2 ./ df_MLE_inc_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

fig_bar_delta_dec_110 = plot()
bar!((df_MLE_dec_p.rho2 ./ df_MLE_dec_p.rho2[1]),label="", color=:blue1, xticks = (1:(S+1), text_list))
# xlabel!("Parameter")
# ylabel!("Relative change")
title!("Patient 110")
ylims!(0,3.0)
plot!(size = (2000, 1000), titlefont=30,xtickfontsize=20,ytickfontsize=30,xguidefontsize=30,yguidefontsize=30,legendfontsize=10,
      margin = 5mm, bottommargin = 12mm,leftmargin = 12mm)

In [ ]:
(VAF_after_2_years_dec[:,1])

In [ ]:
# Plot VAF after 2 years together
l = @layout [a a; a a; a a]
fig_bar_time = plot(fig_bar_time_inc_3, fig_bar_time_dec_3, fig_bar_time_inc_76, fig_bar_time_dec_76, fig_bar_time_inc_110, 
     fig_bar_time_dec_110, layout = l, size = (3000, 2000))

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_bar_time,figpath*"sensitivity_time.png")
savefig(fig_bar_time,figpath*"sensitivity_time.pdf")
savefig(fig_bar_time,figpath*"sensitivity_time.svg")

In [ ]:
# Plot VAF after 2 years together
l = @layout [a a; a a; a a]
fig_bar_VAF = plot(fig_bar_VAF_inc_3, fig_bar_VAF_dec_3, fig_bar_VAF_inc_76, fig_bar_VAF_dec_76, fig_bar_VAF_inc_110, 
     fig_bar_VAF_dec_110, layout = l, size = (3000, 2000))

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_bar_VAF,figpath*"sensitivity_VAF.png")
savefig(fig_bar_VAF,figpath*"sensitivity_VAF.pdf")
savefig(fig_bar_VAF,figpath*"sensitivity_VAF.svg")

In [ ]:
# Plot RMSE
l = @layout [a a; a a; a a]
fig_bar_RMSE = plot(fig_bar_RMSE_inc_3, fig_bar_RMSE_dec_3, fig_bar_RMSE_inc_76, fig_bar_RMSE_dec_76, fig_bar_RMSE_inc_110, 
     fig_bar_RMSE_dec_110, layout = l, size = (3000, 2000))

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_bar_RMSE,figpath*"sensitivity_RMSE.png")
savefig(fig_bar_RMSE,figpath*"sensitivity_RMSE.pdf")
savefig(fig_bar_RMSE,figpath*"sensitivity_RMSE.svg")

In [ ]:
# Plot rho
l = @layout [a a; a a; a a]
fig_bar_rho = plot(fig_bar_rho_inc_3, fig_bar_rho_dec_3, fig_bar_rho_inc_76, fig_bar_rho_dec_76, fig_bar_rho_inc_110, 
     fig_bar_rho_dec_110, layout = l, size = (3000, 2000))

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_bar_rho,figpath*"sensitivity_rho.png")
savefig(fig_bar_rho,figpath*"sensitivity_rho.pdf")
savefig(fig_bar_rho,figpath*"sensitivity_rho.svg")

In [ ]:
# Plot delta
l = @layout [a a; a a; a a]
fig_bar_delta = plot(fig_bar_delta_inc_3, fig_bar_delta_dec_3, fig_bar_delta_inc_76, fig_bar_delta_dec_76, fig_bar_delta_inc_110, 
     fig_bar_delta_dec_110, layout = l, size = (3000, 2000))

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_bar_delta,figpath*"sensitivity_delta.png")
savefig(fig_bar_delta,figpath*"sensitivity_delta.pdf")
savefig(fig_bar_delta,figpath*"sensitivity_delta.svg")

## Optimise multiple models and compare

In [ ]:
# Setup list of models to compare
model_list = ["py0", "py1", "py0py1", "dy0", "dy1IFN", "dy0dy1", "py0dy0", "py0dy1", "dy0py1", "py1dy1",
              "py0py1dy0", "py0py1dy1", "py1dy0dy1", "py0dy0dy1", "py0py1dy0dy1"]

In [ ]:
# Setup list of models to compare
model_list = ["py0", "py1", "py0py1", "dy0", "dy1IFN", "dy0dy1", "py0dy0", "py0dy1", "dy0py1", "py1dy1",
              "py0py1dy0", "py0py1dy1", "py1dy0dy1", "py0dy0dy1", "py0py1dy0dy1"]

# Setup dataframe for storage
df_AIC_BIC = DataFrame(model = String[], Total_AIC = Float64[], Total_BIC = Float64[])

# Set boolean for plot
saveplot = true

# Optimise all models in loop
@showprogress dt=1 desc="Running MLE optimisation for all models..." for k in 1:length(model_list)
    # Choose treatment effect
    effect = model_list[k]
    
    # Extract number of parameterss
    if effect == "sy0dy1" || effect == "py0py1" || effect == "dy0dy1" || effect == "py0dy0" || effect == "py0dy1" || effect == "dy0py1" || effect == "py1dy1"
        n_param = 4
    elseif effect == "dy1" || effect == "py0" || effect == "py1" || effect == "dy0" || effect == "dy1IFN"
        n_param = 3
    elseif effect == "py0py1dy0" || effect == "py0py1dy1" || effect == "py1dy0dy1" || effect == "py0dy0dy1"
        n_param = 5
    elseif effect == "py0py1dy0dy1"
        n_param = 6
    end
    
    # Storage
    if n_param == 4
        df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], sigma = Float64[], 
                           loglik = Float64[], AIC = Float64[], BIC = Float64[])
    elseif n_param == 3
        df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[],
                           AIC = Float64[], BIC = Float64[])
    elseif n_param == 5
        df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], initJAK = Float64[], 
                           sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
    elseif n_param == 6
        df_MLE = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], rho4 = Float64[],
                           initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
    end
        

    # Optimise in loop
    for i in unique(df_D.patientID)
        # Extract relevant data
        pID = i
        df_p = df_D[df_D.patientID .== pID, :]
        
        # Extract number of data points
        n = length(df_p.JAK)

        # Extract VAF - but remove NAN-observations
        pVAF = df_p.JAK
        pVAF = pVAF[.!isnan.(pVAF)]

        # Choose to estimate initial JAK
        estInitJAK = true

        # Instantiate model
        model = model_infer_dynamics(pVAF,df_p,effect,p,master_curve,estInitJAK)

        # Set seed 
        Random.seed!(42)

        # Storage
        if n_param == 4
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], initJAK = Float64[], 
                                    sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 3
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], initJAK = Float64[], 
                                    sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 5
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                    initJAK = Float64[], sigma = Float64[], loglik = Float64[], AIC = Float64[], BIC = Float64[])
        elseif n_param == 6
            df_MLE_temp = DataFrame(patientID = Int64[], rho1 = Float64[], rho2 = Float64[], rho3 = Float64[], 
                                    rho4 = Float64[], initJAK = Float64[], sigma = Float64[], loglik = Float64[], 
                                    AIC = Float64[], BIC = Float64[])
        end

        # Generate a MLE estimate - loop
       for j=1:10
            # Optimise likelihood
            mle_estimate = maximum_likelihood(model, ParticleSwarm(); maxiters = 100, allow_f_increases=true)

            # Setup temporary dataframe and save
            if n_param == 4
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 3
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 5
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            elseif n_param == 6
                df_temp = DataFrame(rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma])
            end
            loglik = loglikelihood(model, df_temp)
            if n_param == 4
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 3
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 5
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            elseif n_param == 6
                df_temp = DataFrame(patientID = pID, rho1 = coef(mle_estimate)[:rho1], rho2 = coef(mle_estimate)[:rho2],
                                    rho3 = coef(mle_estimate)[:rho3], rho4 = coef(mle_estimate)[:rho4],
                                    initJAK = coef(mle_estimate)[:initJAK], sigma = coef(mle_estimate)[:sigma],
                                    loglik = loglik, AIC = 2*n_param-2*loglik, BIC = n_param*log(n)-2*loglik)
            end
            append!(df_MLE_temp, df_temp)
        end

        # Save best MLE
        push!(df_MLE, df_MLE_temp[argmax(df_MLE_temp.loglik),:])   
    
        # Save plots if wanted
        if saveplot
            # Extract relevant data
            pID = i
            df_p = df_D[df_D.patientID .== pID, :]
            
            # Extract latest data point
            maxDays = maximum(df_p.days)

            # Extract VAF - but remove NAN-observations
            pVAF = df_p.JAK
            pVAF = pVAF[.!isnan.(pVAF)]

            # Choose to estimate initial JAK
            estInitJAK = true

            # Choose time for calculating VAF
            pDays = collect(1:11*365)
            pDays2 = df_p.days[.!isnan.(df_p.JAK)]

            # Load standard parameters
            include("model_default_param.jl")

            # Extract MLE estimate
            df_MLE_p = df_MLE[df_MLE.patientID .== pID,:]
            if n_param == 4
                rho = [df_MLE_p.rho1[1], df_MLE_p.rho2[1]]
            elseif n_param == 3
                rho = df_MLE_p.rho1[1]
            elseif n_param == 5
                rho = [df_MLE_p.rho1[1], df_MLE_p.rho2[1], df_MLE_p.rho3[1]]
            elseif n_param == 6
                rho = [df_MLE_p.rho1[1], df_MLE_p.rho2[1], df_MLE_p.rho3[1], df_MLE_p.rho4[1]]
            end
            initJAK = df_MLE_p.initJAK[1]

            # Calculate VAF using function
            VAF_MLE, sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
            VAF_MLE_points, sol = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)

            # Calculate RMSE
            RMSE = rmsd(VAF_MLE_points,pVAF)

            # Plot results
            figVAF = plot(pDays[pDays.<=maxDays]/30.4, VAF_MLE[pDays.<=maxDays]*100, linewidth = 3, label = L"JAK2"*" VAF - MLE", colour= :green1)
            plot!(pDays[pDays.>maxDays]/30.4, VAF_MLE[pDays.>maxDays]*100, linewidth = 3, label = L"JAK2"*" VAF - MLE", colour= :green1, linestyle = :dash)
            # plot!(pDays/30.4, VAF_true*100, linewidth = 3, label = L"JAK2"*" VAF - True", colour = :darkgoldenrod2)
            scatter!(df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=4)
            if n_param == 4
                title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho_{p_{y_0}}="
                       *"$(round(df_MLE_p.rho1[1]; sigdigits=3)), " * L"\rho_{p_{y_1}}="*"$(round(df_MLE_p.rho2[1]; sigdigits=3))")
            elseif n_param == 3
                title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho_{p_{y_0}}="
                       *"$(round(df_MLE_p.rho1[1]; sigdigits=3))")
            end
            xlabel!(L"t"*"/months")
            ylabel!(L"JAK2"*" VAF/%")
            t_end = 125
            xlims!(0, t_end)
            ylims!(0,100)
            plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
                legend = :none)
            figIFN = plot(df_p.days/30.4, df_p.IFN, line =:steppost, linewidth = 3, label = "", colour= :blue1)
            plot!([maxDays/30.4,120], [df_p.IFN[end],df_p.IFN[end]], line =:steppost, linewidth = 3, label = "", colour= :blue1,
                   linestyle = :dash)
            title!("IFN Dose for Patient $(pID)")
            xlabel!(L"t"*"/months")
            ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
            xlims!(0, t_end)
            ylims!(0, 20)
            plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10)
            figcombined = plot(figVAF,figIFN, layout=grid(2,1, heights=(7/10, 3/10)), link =:x, size = (700, 500), margin=5mm)

            # Save figure
            figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MLE/$(effect)/"
            savefig(figcombined,figpath*"MLE_$(effect)_p$(pID).png")
            savefig(figcombined,figpath*"MLE_$(effect)_p$(pID).pdf")
            savefig(figcombined,figpath*"MLE_$(effect)_p$(pID).svg")
        end
    end
    
    # Write results to CSV file
    path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y Data - MLE files/MLE_$(effect).csv"
    CSV.write(path, df_MLE)
    
    # Save sum of BICs in dataframe
    df_temp = DataFrame(model = effect, Total_AIC = sum(df_MLE.AIC), Total_BIC = sum(df_MLE.BIC))
    append!(df_AIC_BIC, df_temp)
end

# Save df_AIC_BIC
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y Data - MLE files/Total_AIC_BIC.csv"
CSV.write(path, df_AIC_BIC)

In [ ]:
# Load df_AIC_BIC
df_AIC_BIC = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y Data - MLE files/Total_AIC_BIC.csv"
df_AIC_BIC = CSV.read(path, df_AIC_BIC)

In [ ]:
# Calculate relative likelihood - does not make sense for the sum of likelihoods
df_AIC_BIC.relLik = exp.((minimum(df_AIC_BIC.Total_AIC).-df_AIC_BIC.Total_AIC)/2)
df_AIC_BIC

In [ ]:
# Storage
df_AIC_all = DataFrame(patientID = unique(df_D.patientID))
df_BIC_all = DataFrame(patientID = unique(df_D.patientID))
    
# Extract AIC and BIC for a single patient
for k in 1:length(model_list)
     # Choose treatment effect
    effect = model_list[k]

    # Load results
    df_MLE = DataFrame
    path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y Data - MLE files/MLE_$(effect).csv"
    df_MLE = CSV.read(path, df_MLE)
    
    temp_AIC = []
    temp_BIC = []

    for i in unique(df_D.patientID)
        # Set pID
        pID = i

        # Extract AIC and BIC
        df_MLE_temp = df_MLE[df_MLE.patientID .== pID,:]
        append!(temp_AIC,df_MLE_temp.AIC)
        append!(temp_BIC,df_MLE_temp.BIC)
    end
    
    # Put in dataframe
    df_AIC_all[!, "$(effect)"] = temp_AIC
    df_BIC_all[!, "$(effect)"] = temp_BIC
end
df_AIC_all

In [ ]:
# Storage
df_relLik_all = copy(df_AIC_all)
df_relLik_BIC_all = copy(df_BIC_all)

for i=1:size(df_AIC_all)[1]
    for j=2:size(df_AIC_all)[2]
        # Calculate relative likelihood and save
        df_relLik_all[i,j] = exp((minimum(df_AIC_all[i,:])-df_AIC_all[i,j])/2)
        df_relLik_BIC_all[i,j] = exp((minimum(df_BIC_all[i,:])-df_BIC_all[i,j])/2)
    end
end
df_relLik_all

In [ ]:
# Sum of relative likelihoods
sum_rel_lik = vec(sum(Matrix(df_relLik_all[:,2:end]),dims=1))
scatter(sum_rel_lik, markercolor=:red, markersize=4, xticks = (1:15, model_list), label = "")
title!("\nSum of relative likelihoods")
xlabel!("Model")
ylabel!("Relative likelihoods")
# xlims!(0.5, 10.5)
plot!(titlefont=20,xtickfontsize=7,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,size = (1000, 700),
      margin=5mm)

In [ ]:
# x ticks
model_list_plot = [L"p_{y_0}", L"p_{y_1}", L"p_{y_0}, "*L"p_{y_1}", L"d_{y_0}", L"d_{y_1}", L"d_{y_0}, "*L"d_{y_1}", 
                   L"p_{y_0}, "*L"d_{y_0}", L"p_{y_0}, "*L"d_{y_1}", L"d_{y_0}, "*L"p_{y_1}", L"p_{y_1}, "*L"d_{y_1}",
                   L"p_{y_0}, "*L"p_{y_1}, "*L"d_{y_0}", L"p_{y_0}, "*L"p_{y_1}, "*L"d_{y_1}",
                   L"p_{y_1}, "*L"d_{y_0}, "*L"d_{y_1}", L"p_{y_0}, "*L"d_{y_0}, "*L"d_{y_1}",
                   "All 4"] 

# Number of "good models" (relative likelihood greater than a threshold)
threshold = 0.2
sum_good_model = vec(sum(Matrix(df_relLik_all[:,2:end]).>threshold,dims=1))
fig_relLik = scatter(sum_good_model, markercolor=:red, markersize=4, xticks = (1:15, model_list_plot), label = "")
title!("\nNumber of patients for which the model\n has a relative likelihood above 0.2")
xlabel!("Model")
ylabel!("Number of patients")
xlims!(0.5, 15.5)
ylims!(0, 64)
plot!(titlefont=20,xtickfontsize=7,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,size = (1000, 700),
      margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MLE/"
savefig(fig_relLik,figpath*"relLik_comparison.png")
savefig(fig_relLik,figpath*"relLik_comparison.pdf")
savefig(fig_relLik,figpath*"relLik_comparison.svg")

In [ ]:
# Number of best models
threshold = 1.0
sum_best_model = vec(sum(Matrix(df_relLik_all[:,2:end]).>=threshold,dims=1))
scatter(sum_best_model, markercolor=:red, markersize=4, xticks = (1:10, model_list), label = "")
title!("\nNumber of patients for whom the model has the highest AIC")
xlabel!("Model")
ylabel!("Number of patients")
xlims!(0.5, 10.5)
ylims!(0, 17)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,size = (1000, 700),
      margin=5mm)

In [ ]:
# Number of best models - BIC
threshold = 1.0
sum_best_model = vec(sum(Matrix(df_relLik_BIC_all[:,2:end]).>=threshold,dims=1))
scatter(sum_best_model, markercolor=:red, markersize=4, xticks = (1:10, model_list), label = "")
title!("\nNumber of patients for which the model has the highest BIC")
xlabel!("Model")
ylabel!("Number of patients")
xlims!(0.5, 10.5)
ylims!(0, 17)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,size = (1000, 700),
      margin=5mm)

In [ ]:
# Setup list of models to compare
model_list_plot = [L"p_{y_0}", L"p_{y_1}", L"p_{y_0}, "*L"p_{y_1}", L"d_{y_0}", L"d_{y_1}", L"d_{y_0}, "*L"d_{y_1}", 
                   L"p_{y_0}, "*L"d_{y_0}", L"p_{y_0}, "*L"d_{y_1}", L"d_{y_0}, "*L"p_{y_1}", L"p_{y_1}, "*L"d_{y_1}",
                   L"p_{y_0}, "*L"p_{y_1}, "*L"d_{y_0}", L"p_{y_0}, "*L"p_{y_1}, "*L"d_{y_1}",
                   L"p_{y_1}, "*L"d_{y_0}, "*L"d_{y_1}", L"p_{y_0}, "*L"d_{y_0}, "*L"d_{y_1}",
                   "All 4"] 

# Plot results
fig_AIC = scatter(df_AIC_BIC.Total_AIC,label="", markercolor=:red, markersize=4, xticks = (1:15, model_list_plot))
title!("Model AIC Comparison")
xlabel!("Model")
ylabel!("Sum of AIC")
xlims!(0.5, 15.5)
ylims!(-2000,0)
plot!(titlefont=20,xtickfontsize=7,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,size = (1000, 700),
      margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MLE/"
savefig(fig_AIC,figpath*"AIC_comparison.png")
savefig(fig_AIC,figpath*"AIC_comparison.pdf")
savefig(fig_AIC,figpath*"AIC_comparison.svg")

In [ ]:
# Setup list of models to compare
model_list_plot = [L"p_{y_0}", L"p_{y_1}", L"p_{y_0}, "*L"p_{y_1}", L"d_{y_0}", L"d_{y_1}", L"d_{y_0}, "*L"d_{y_1}", 
                   L"p_{y_0}, "*L"d_{y_0}", L"p_{y_0}, "*L"d_{y_1}", L"d_{y_0}, "*L"p_{y_1}", L"p_{y_1}, "*L"d_{y_1}",
                   L"p_{y_0}, "*L"p_{y_1}, "*L"d_{y_0}", L"p_{y_0}, "*L"p_{y_1}, "*L"d_{y_1}",
                   L"p_{y_1}, "*L"d_{y_0}, "*L"d_{y_1}", L"p_{y_0}, "*L"d_{y_0}, "*L"d_{y_1}",
                   "All 4"] 

# Plot results
fig_BIC = scatter(df_AIC_BIC.Total_BIC,label="", markercolor=:red, markersize=4, xticks = (1:15, model_list_plot))
title!("Model BIC Comparison")
xlabel!("Model")
ylabel!("Sum of BIC")
xlims!(0.5, 15.5)
ylims!(-2000,0)
plot!(titlefont=20,xtickfontsize=7,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10,size = (1000, 700),
      margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MLE/"
savefig(fig_BIC,figpath*"BIC_comparison.png")
savefig(fig_BIC,figpath*"BIC_comparison.pdf")
savefig(fig_BIC,figpath*"BIC_comparison.svg")